In [ ]:
import requests
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from osgeo import gdal

In [ ]:
# Get map data from OpenTopography, check response
%%time
resp = requests.get(
    'https://portal.opentopography.org/API'
    '/globaldem?demtype=SRTMGL3'
    '&south=45.55'
    '&north=49'
    '&west=-124.77'
    '&east=-116.92'
    '&outputFormat=GTiff')
resp

In [ ]:
# Save response (map data) as raster
with open('map.tiff', 'wb') as f:
    f.write(resp.content)

In [ ]:
data = Image.open('map.tiff') # Open map data as image
np_map = np.maximum(0, np.array(data)) ** 0.8 # Convert to array, remove negative values, nerf large mountains
pad_val = 1000
pad = (pad_val,pad_val),(pad_val,pad_val)
np_map = np.pad(np_map, pad, 'constant', constant_values=np_map.min()) # Create a border
np_map[np_map <= 0] = -200 # Drop sea level (for more visible coastline)

In [ ]:
# Peek map
plt.figure(figsize=(20,10))
plt.imshow(np_map, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
np_map.min(), np_map.max()

In [ ]:
# Convert to int array for Blender
norm_map = ((np_map - np_map.min()) * 65000 / (np_map.max() - np_map.min())).astype('uint16') 
plt.figure(figsize=(20,10))
plt.imshow(norm_map, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
Image.fromarray(norm_map).save('map_norm.tiff') # Save map for use in blender

In [ ]:
norm_map.shape # Dimensions for use in Blender